notebook1
## PART 1. Document retrieval

# 清理舊測試資料

In [1]:
# !rm -rf data/hanlp_con_test_results.pkl
# !rm -rf data/test_doc5.jsonl
!rm -rf data/test_doc5sent5.jsonl
!rm -rf data/test_doc5sent5.pkl

In [1]:
step2_model = 'step2_new_upload'
step3_model = 'step3_base_upload'


Prepare the environment and import all library we need

In [2]:
# built-in libs
import json
import pickle
import re
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Set, Tuple, Union

# 3rd party libs
import hanlp
import opencc
import pandas as pd
import wikipedia
from hanlp.components.pipeline import Pipeline
from pandarallel import pandarallel

# our own libs


def load_json(file_path: Union[Path, str]) -> pd.DataFrame:
    """jsonl_to_df read jsonl file and return a pandas DataFrame.

    Args:
        file_path (Union[Path, str]): The jsonl file path.

    Returns:
        pd.DataFrame: The jsonl file content.

    Example:
        >>> read_jsonl_file("data/train.jsonl")
               id            label  ... predicted_label                                      evidence_list
        0    3984          refutes  ...         REFUTES  [城市規劃是城市建設及管理的依據 ， 位於城市管理之規劃 、 建設 、 運作三個階段之首 ，...
        ..    ...              ...  ...             ...                                                ...
        945  3042         supports  ...         REFUTES  [北歐人相傳每當雷雨交加時就是索爾乘坐馬車出來巡視 ， 因此稱呼索爾為 “ 雷神 ” 。, ...

        [946 rows x 10 columns]
    """
    with open(file_path, "r", encoding="utf8") as json_file:
        json_list = list(json_file)

    return [json.loads(json_str) for json_str in json_list]


pandarallel.initialize(progress_bar=True, verbose=0, nb_workers=10)
wikipedia.set_lang("zh")

Preload the data.

In [3]:
TEST_DATA = load_json("data/public_test.jsonl")
CONVERTER_T2S = opencc.OpenCC("t2s.json")
CONVERTER_S2T = opencc.OpenCC("s2t.json")

Data class for type hinting

In [4]:
@dataclass
class Claim:
    data: str

@dataclass
class AnnotationID:
    id: int

@dataclass
class EvidenceID:
    id: int

@dataclass
class PageTitle:
    title: str

@dataclass
class SentenceID:
    id: int

@dataclass
class Evidence:
    data: List[List[Tuple[AnnotationID, EvidenceID, PageTitle, SentenceID]]]

### Helper function

For the sake of consistency, we convert traditional to simplified Chinese first before converting it back to traditional Chinese.  This is due to some errors occuring when converting traditional to traditional Chinese.

In [5]:
def do_st_corrections(text: str) -> str:
    simplified = CONVERTER_T2S.convert(text)

    return CONVERTER_S2T.convert(simplified)

We use constituency parsing to separate part of speeches or so called constituent to extract noun phrases.  In the later stages, we will use the noun phrases as the query to search for relevant documents.  

In [6]:
def get_nps_hanlp(
    predictor: Pipeline,
    d: Dict[str, Union[int, Claim, Evidence]],
) -> List[str]:
    claim = d["claim"]
    tree = predictor(claim)["con"]
    nps = [
        do_st_corrections("".join(subtree.leaves()))
        for subtree in tree.subtrees(lambda t: t.label() == "NP")
    ]

    return nps

Precision refers to how many related documents are retrieved.  Recall refers to how many relevant documents are retrieved.  

The default amount of documents retrieved is at most five documents.  This `num_pred_doc` can be adjusted based on your objective.  Save data in jsonl format.

In [7]:
def save_doc(
    data: List[Dict[str, Union[int, Claim, Evidence]]],
    predictions: pd.Series,
    mode: str = "train",
    num_pred_doc: int = 5,
) -> None:
    with open(
        f"data/{mode}_doc{num_pred_doc}.jsonl",
        "w",
        encoding="utf8",
    ) as f:
        for i, d in enumerate(data):
            d["predicted_pages"] = list(predictions.iloc[i])
            f.write(json.dumps(d, ensure_ascii=False) + "\n")

### Main function for document retrieval

In [8]:
def get_pred_pages(series_data: pd.Series) -> Set[Dict[int, str]]:
    results = []
    tmp_muji = []
    # wiki_page: its index showned in claim
    mapping = {}
    claim = series_data["claim"]
    nps = series_data["hanlp_results"]
    first_wiki_term = []

    for i, np in enumerate(nps):
        # Simplified Traditional Chinese Correction
        wiki_search_results = [
            do_st_corrections(w) for w in wikipedia.search(np)
        ]

        # Remove the wiki page's description in brackets
        wiki_set = [re.sub(r"\s\(\S+\)", "", w) for w in wiki_search_results]
        wiki_df = pd.DataFrame({
            "wiki_set": wiki_set,
            "wiki_results": wiki_search_results
        })

        # Elements in wiki_set --> index
        # Extracting only the first element is one way to avoid extracting
        # too many of the similar wiki pages
        grouped_df = wiki_df.groupby("wiki_set", sort=False).first()
        candidates = grouped_df["wiki_results"].tolist()
        # muji refers to wiki_set
        muji = grouped_df.index.tolist()

        for prefix, term in zip(muji, candidates):
            if prefix not in tmp_muji:
                matched = False

                # Take at least one term from the first noun phrase
                if i == 0:
                    first_wiki_term.append(term)

                # Walrus operator :=
                # https://docs.python.org/3/whatsnew/3.8.html#assignment-expressions
                # Through these filters, we are trying to figure out if the term
                # is within the claim
                if (((new_term := term) in claim) or
                    ((new_term := term.replace("·", "")) in claim) or
                    ((new_term := term.split(" ")[0]) in claim) or
                    ((new_term := term.replace("-", " ")) in claim)):
                    matched = True

                elif "·" in term:
                    splitted = term.split("·")
                    for split in splitted:
                        if (new_term := split) in claim:
                            matched = True
                            break

                if matched:
                    # post-processing
                    term = term.replace(" ", "_")
                    term = term.replace("-", "")
                    results.append(term)
                    mapping[term] = claim.find(new_term)
                    tmp_muji.append(new_term)

    # 5 is a hyperparameter
    if len(results) > 5:
        assert -1 not in mapping.values()
        results = sorted(mapping, key=mapping.get)
    elif len(results) < 1:
        results = first_wiki_term

    return set(results)

### Step 1. Get noun phrases from hanlp consituency parsing tree

Setup [HanLP](https://github.com/hankcs/HanLP) predictor (1 min)

In [9]:
predictor = (hanlp.pipeline().append(
    hanlp.load("MSR_TOK_ELECTRA_BASE_CRF"),
    output_key="tok",
).append(
    hanlp.load("CTB9_CON_ELECTRA_SMALL"),
    output_key="con",
    input_key="tok",
))
# FINE_ELECTRA_SMALL_ZH

We will skip this process which for creating parsing tree when demo on class

### Step 3. Repeat the same process on test set
Create parsing tree

In [10]:
hanlp_test_file = f"data/hanlp_con_test_results.pkl"
if Path(hanlp_test_file).exists():
    with open(hanlp_test_file, "rb") as f:
        hanlp_results = pickle.load(f)
else:
    hanlp_results = [get_nps_hanlp(predictor, d) for d in TEST_DATA]
    with open(hanlp_test_file, "wb") as f:
        pickle.dump(hanlp_results, f)

Get pages via wiki online api

In [11]:
test_doc_path = f"data/test_doc5.jsonl"
if Path(test_doc_path).exists():
    with open(test_doc_path, "r", encoding="utf8") as f:
        test_results = pd.Series(
            [set(json.loads(line)["predicted_pages"]) for line in f])
else:
    test_df = pd.DataFrame(TEST_DATA)
    test_df.loc[:, "hanlp_results"] = hanlp_results
    test_results = test_df.parallel_apply(get_pred_pages, axis=1)
    save_doc(TEST_DATA, test_results, mode="test")
print(test_results.head())

0        {電磁學, 物, 光學顯微鏡, 肉眼, 顯微鏡}
1                         {蠶, 昆蟲}
2        {波蘭, 人, 西部, 0, 土地, 綠山城縣}
3               {魂斷藍橋, 配角, 藍橋, 橋}
4    {文學, 劇情片, 侯孝賢, 金馬獎, 文言文, 電影}
dtype: object


notebook2
## PART 2. Sentence retrieval

In [12]:
import torch

Import some libs

In [2]:
# built-in libs
from pathlib import Path
from typing import Dict, List, Set, Tuple, Union

# third-party libs
import numpy as np
import pandas as pd
from pandarallel import pandarallel
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
# from transformers import (
#     AutoModelForSequenceClassification,
#     AutoTokenizer,
#     get_scheduler,
# )
# local libs
from typing import Dict, Union
from pathlib import Path
import json
import re
import pandas as pd
# import torch
# from transformers import get_scheduler


# Helper functions


def load_json(file_path: Union[Path, str]) -> pd.DataFrame:
    """jsonl_to_df read jsonl file and return a pandas DataFrame.

    Args:
        file_path (Union[Path, str]): The jsonl file path.

    Returns:
        pd.DataFrame: The jsonl file content.

    Example:
        >>> read_jsonl_file("data/train.jsonl")
               id            label  ... predicted_label                                      evidence_list
        0    3984          refutes  ...         REFUTES  [城市規劃是城市建設及管理的依據 ， 位於城市管理之規劃 、 建設 、 運作三個階段之首 ，...
        ..    ...              ...  ...             ...                                                ...
        945  3042         supports  ...         REFUTES  [北歐人相傳每當雷雨交加時就是索爾乘坐馬車出來巡視 ， 因此稱呼索爾為 “ 雷神 ” 。, ...

        [946 rows x 10 columns]
    """
    with open(file_path, "r", encoding="utf8") as json_file:
        json_list = list(json_file)

    return [json.loads(json_str) for json_str in json_list]


def jsonl_dir_to_df(dir_path: Union[Path, str]) -> pd.DataFrame:
    """jsonl_dir_to_df read jsonl dir and return a pandas DataFrame.

    This function will read all jsonl files in the dir_path and concat them.

    Args:
        dir_path (Union[Path, str]): The jsonl dir path.

    Returns:
        pd.DataFrame: The jsonl dir content.

    Example:
        >>> read_jsonl_dir("data/extracted_dir/")
               id            label  ... predicted_label                                      evidence_list
        0    3984          refutes  ...         REFUTES  [城市規劃是城市建設及管理的依據 ， 位於城市管理之規劃 、 建設 、 運作三個階段之首 ，...
        ..    ...              ...  ...             ...                                                ...
        945  3042         supports  ...         REFUTES  [北歐人相傳每當雷雨交加時就是索爾乘坐馬車出來巡視 ， 因此稱呼索爾為 “ 雷神 ” 。, ...

        [946 rows x 10 columns]
    """
    print(f"Reading and concatenating jsonl files in {dir_path}")
    return pd.concat(
        [pd.DataFrame(load_json(file)) for file in Path(dir_path).glob("*.jsonl")]
    )


def generate_evidence_to_wiki_pages_mapping(
    wiki_pages: pd.DataFrame,
) -> Dict[str, Dict[int, str]]:
    """generate_wiki_pages_dict generate a mapping from evidence to wiki pages by evidence id.

    Args:
        wiki_pages (pd.DataFrame): The wiki pages dataframe
        cache(Union[Path, str], optional): The cache file path. Defaults to None.
            If cache is None, return the result directly.

    Returns:
        pd.DataFrame:
    """

    def make_dict(x):
        result = {}
        sentences = re.split(r"\n(?=[0-9])", x)
        for sent in sentences:
            splitted = sent.split("\t")
            if len(splitted) < 2:
                # Avoid empty articles
                return result
            result[splitted[0]] = splitted[1]
        return result

    # copy wiki_pages
    wiki_pages = wiki_pages.copy()

    # generate parse mapping
    print("Generate parse mapping")
    wiki_pages["evidence_map"] = wiki_pages["lines"].parallel_map(make_dict)
    # generate id to evidence_map mapping
    print("Transform to id to evidence_map mapping")
    mapping = dict(
        zip(
            wiki_pages["id"].to_list(),
            wiki_pages["evidence_map"].to_list(),
        )
    )
    # release memory
    del wiki_pages
    return mapping


pandarallel.initialize(progress_bar=True, verbose=0, nb_workers=10)

Global variable

In [7]:
SEED = 42

TEST_DATA = load_json("data/public_test.jsonl")

LABEL2ID: Dict[str, int] = {
    "supports": 0,
    "refutes": 1,
    "NOT ENOUGH INFO": 2,
}
ID2LABEL: Dict[int, str] = {v: k for k, v in LABEL2ID.items()}
# GT means Ground Truth


Preload wiki database (1 min)

In [8]:
wiki_pages = jsonl_dir_to_df("data/wiki-pages")
mapping = generate_evidence_to_wiki_pages_mapping(wiki_pages)
del wiki_pages

Reading and concatenating jsonl files in data/wiki-pages
Generate parse mapping


Transform to id to evidence_map mapping


### Helper function

Calculate precision for sentence retrieval

In [9]:
def evidence_macro_precision(
    instance: Dict,
    top_rows: pd.DataFrame,
) -> Tuple[float, float]:
    """Calculate precision for sentence retrieval
    This function is modified from fever-scorer.
    https://github.com/sheffieldnlp/fever-scorer/blob/master/src/fever/scorer.py

    Args:
        instance (dict): a row of the dev set (dev.jsonl) of test set (test.jsonl)
        top_rows (pd.DataFrame): our predictions with the top probabilities

        IMPORTANT!!!
        instance (dict) should have the key of `evidence`.
        top_rows (pd.DataFrame) should have a column `predicted_evidence`.

    Returns:
        Tuple[float, float]:
        [1]: relevant and retrieved (numerator of precision)
        [2]: retrieved (denominator of precision)
    """
    this_precision = 0.0
    this_precision_hits = 0.0

    # Return 0, 0 if label is not enough info since not enough info does not
    # contain any evidence.
    if instance["label"].upper() != "NOT ENOUGH INFO":
        # e[2] is the page title, e[3] is the sentence index
        all_evi = [[e[2], e[3]]
                   for eg in instance["evidence"]
                   for e in eg
                   if e[3] is not None]
        claim = instance["claim"]
        predicted_evidence = top_rows[top_rows["claim"] ==
                                      claim]["predicted_evidence"].tolist()

        for prediction in predicted_evidence:
            if prediction in all_evi:
                this_precision += 1.0
            this_precision_hits += 1.0

        return (this_precision /
                this_precision_hits) if this_precision_hits > 0 else 1.0, 1.0

    return 0.0, 0.0

Calculate recall for sentence retrieval

In [10]:
def evidence_macro_recall(
    instance: Dict,
    top_rows: pd.DataFrame,
) -> Tuple[float, float]:
    """Calculate recall for sentence retrieval
    This function is modified from fever-scorer.
    https://github.com/sheffieldnlp/fever-scorer/blob/master/src/fever/scorer.py

    Args:
        instance (dict): a row of the dev set (dev.jsonl) of test set (test.jsonl)
        top_rows (pd.DataFrame): our predictions with the top probabilities

        IMPORTANT!!!
        instance (dict) should have the key of `evidence`.
        top_rows (pd.DataFrame) should have a column `predicted_evidence`.

    Returns:
        Tuple[float, float]:
        [1]: relevant and retrieved (numerator of recall)
        [2]: relevant (denominator of recall)
    """
    # We only want to score F1/Precision/Recall of recalled evidence for NEI claims
    if instance["label"].upper() != "NOT ENOUGH INFO":
        # If there's no evidence to predict, return 1
        if len(instance["evidence"]) == 0 or all(
            [len(eg) == 0 for eg in instance]):
            return 1.0, 1.0

        claim = instance["claim"]

        predicted_evidence = top_rows[top_rows["claim"] ==
                                      claim]["predicted_evidence"].tolist()

        for evidence_group in instance["evidence"]:
            evidence = [[e[2], e[3]] for e in evidence_group]
            if all([item in predicted_evidence for item in evidence]):
                # We only want to score complete groups of evidence. Incomplete
                # groups are worthless.
                return 1.0, 1.0
        return 0.0, 1.0
    return 0.0, 0.0

Calculate the scores of sentence retrieval

In [11]:
def evaluate_retrieval(
    probs: np.ndarray,
    df_evidences: pd.DataFrame,
    ground_truths: pd.DataFrame,
    top_n: int = 5,
    cal_scores: bool = True,
    save_name: str = None,
) -> Dict[str, float]:
    """Calculate the scores of sentence retrieval

    Args:
        probs (np.ndarray): probabilities of the candidate retrieved sentences
        df_evidences (pd.DataFrame): the candiate evidence sentences paired with claims
        ground_truths (pd.DataFrame): the loaded data of dev.jsonl or test.jsonl
        top_n (int, optional): the number of the retrieved sentences. Defaults to 2.

    Returns:
        Dict[str, float]: F1 score, precision, and recall
    """
    df_evidences["prob"] = probs
    top_rows = (
        df_evidences.groupby("claim").apply(
        lambda x: x.nlargest(top_n, "prob"))
        .reset_index(drop=True)
    )

    if cal_scores:
        macro_precision = 0
        macro_precision_hits = 0
        macro_recall = 0
        macro_recall_hits = 0

        for i, instance in enumerate(ground_truths):
            macro_prec = evidence_macro_precision(instance, top_rows)
            macro_precision += macro_prec[0]
            macro_precision_hits += macro_prec[1]

            macro_rec = evidence_macro_recall(instance, top_rows)
            macro_recall += macro_rec[0]
            macro_recall_hits += macro_rec[1]

        pr = (macro_precision /
              macro_precision_hits) if macro_precision_hits > 0 else 1.0
        rec = (macro_recall /
               macro_recall_hits) if macro_recall_hits > 0 else 0.0
        f1 = 2.0 * pr * rec / (pr + rec)

    if save_name is not None:
        # write doc7_sent5 file
        with open(f"data/{save_name}", "w") as f:
            for instance in ground_truths:
                claim = instance["claim"]
                predicted_evidence = top_rows[
                    top_rows["claim"] == claim]["predicted_evidence"].tolist()
                instance["predicted_evidence"] = predicted_evidence
                f.write(json.dumps(instance, ensure_ascii=False) + "\n")

    if cal_scores:
        return {"F1 score": f1, "Precision": pr, "Recall": rec}

Inference script to get probabilites for the candidate evidence sentences

AicupTopkEvidenceBERTDataset class for AICUP dataset with top-k evidence sentences

### Main function for sentence retrieval

In [20]:
def pair_with_wiki_sentences_eval(
    mapping: Dict[str, Dict[int, str]],
    df: pd.DataFrame,
    is_testset: bool = False,
) -> pd.DataFrame:
    """Only for creating dev and test sentences."""
    claims = []
    sentences = []
    evidence = []
    idx = []
    predicted_evidence = []

    # negative
    for i in range(len(df)):
        # if df["label"].iloc[i] == "NOT ENOUGH INFO":
        #     continue
        claim = df["claim"].iloc[i]

        predicted_pages = df["predicted_pages"][i]
        for page in predicted_pages:
            page = page.replace(" ", "_")
            try:
                page_sent_id_pairs = [(page, k) for k in mapping[page]]
            except KeyError:
                # print(f"{page} is not in our Wiki db.")
                continue

            for page_name, sentence_id in page_sent_id_pairs:
                text = mapping[page][sentence_id]
                if text != "":
                    claims.append(claim)
                    sentences.append(text.replace(" ",""))
                    idx.append(page)
                    if not is_testset:
                        evidence.append(df["evidence"].iloc[i])
                    predicted_evidence.append([page_name, int(sentence_id)])

    return pd.DataFrame({
        "claim": claims,
        "text": sentences,
        "idx": idx,
        "evidence": evidence if not is_testset else None,
        "predicted_evidence": predicted_evidence,
    })

### Step 1. Setup training environment

Hyperparams

In [21]:
#@title  { display-mode: "form" }
TOP_N = 5  #@param {type:"integer"}

Experiment Directory

### Step 2. Predict

Validation part (15 mins)

### Step 4. Check on our test data
(5 min)

In [22]:
from autogluon.multimodal import MultiModalPredictor
predictor = MultiModalPredictor.load(step2_model)

/opt/conda/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
Load pretrained checkpoint: /workspace/main/AICUP-2023-Truth-main/step2_new_upload/model.ckpt


In [23]:
test_data = load_json("data/test_doc5.jsonl")

test_evidences = pair_with_wiki_sentences_eval(
    mapping,
    pd.DataFrame(test_data),
    is_testset=True,
)
test_evidences.head()



,claim,text,idx,evidence,predicted_evidence
0,光學顯微鏡是以電磁學原理來將不可見或難見的微小物放大至肉眼可見的儀器。,光學顯微鏡（Opticalmicroscope、Lightmicroscope）是一種利用光...,光學顯微鏡,None,"[光學顯微鏡, 0]"
1,光學顯微鏡是以電磁學原理來將不可見或難見的微小物放大至肉眼可見的儀器。,由物體入射的光被至少兩個光學系統（物鏡和目鏡）放大。,光學顯微鏡,None,"[光學顯微鏡, 3]"
2,光學顯微鏡是以電磁學原理來將不可見或難見的微小物放大至肉眼可見的儀器。,首先物鏡產生一個被放大實像，人眼通過作用相當於放大鏡的目鏡觀察這個已經被放大了的實像。,光學顯微鏡,None,"[光學顯微鏡, 4]"
3,光學顯微鏡是以電磁學原理來將不可見或難見的微小物放大至肉眼可見的儀器。,一般的光學顯微鏡有多個可以替換的物鏡，這樣觀察者可以按需要更換放大倍數，也就是增加放大倍率，...,光學顯微鏡,None,"[光學顯微鏡, 5]"
4,光學顯微鏡是以電磁學原理來將不可見或難見的微小物放大至肉眼可見的儀器。,這些物鏡一般被安置在一個可以轉動的物鏡盤上，轉動物鏡盤就可以使不同的物鏡方便地進入光路，物鏡...,光學顯微鏡,None,"[光學顯微鏡, 6]"


In [24]:
print("Start predicting the test data")
probs = predictor.predict_proba(test_evidences)
probs = probs.to_numpy()
second_values = [p[1] for p in probs]

Start predicting the test data


Predicting: 0it [00:00, ?it/s]

In [25]:
evaluate_retrieval(
    probs=second_values,
    df_evidences=test_evidences,
    ground_truths=test_data,
    top_n=TOP_N,
    cal_scores=False,
    save_name=f"test_doc5sent{TOP_N}.jsonl",
)

In [26]:
test_evidences.head(15)

,claim,text,idx,evidence,predicted_evidence,prob
0,光學顯微鏡是以電磁學原理來將不可見或難見的微小物放大至肉眼可見的儀器。,光學顯微鏡（Opticalmicroscope、Lightmicroscope）是一種利用光...,光學顯微鏡,None,"[光學顯微鏡, 0]",0.964085
1,光學顯微鏡是以電磁學原理來將不可見或難見的微小物放大至肉眼可見的儀器。,由物體入射的光被至少兩個光學系統（物鏡和目鏡）放大。,光學顯微鏡,None,"[光學顯微鏡, 3]",0.221519
2,光學顯微鏡是以電磁學原理來將不可見或難見的微小物放大至肉眼可見的儀器。,首先物鏡產生一個被放大實像，人眼通過作用相當於放大鏡的目鏡觀察這個已經被放大了的實像。,光學顯微鏡,None,"[光學顯微鏡, 4]",0.009423
3,光學顯微鏡是以電磁學原理來將不可見或難見的微小物放大至肉眼可見的儀器。,一般的光學顯微鏡有多個可以替換的物鏡，這樣觀察者可以按需要更換放大倍數，也就是增加放大倍率，...,光學顯微鏡,None,"[光學顯微鏡, 5]",0.001495
4,光學顯微鏡是以電磁學原理來將不可見或難見的微小物放大至肉眼可見的儀器。,這些物鏡一般被安置在一個可以轉動的物鏡盤上，轉動物鏡盤就可以使不同的物鏡方便地進入光路，物鏡...,光學顯微鏡,None,"[光學顯微鏡, 6]",0.000617
5,光學顯微鏡是以電磁學原理來將不可見或難見的微小物放大至肉眼可見的儀器。,十八世紀，光學顯微鏡的放大倍率已經提高到了1000倍，使人們能用眼睛看清微生物體的形態、大小...,光學顯微鏡,None,"[光學顯微鏡, 9]",0.086170
6,光學顯微鏡是以電磁學原理來將不可見或難見的微小物放大至肉眼可見的儀器。,直到物理學家發現了放大倍率與解析度之間的規律，人們才知道光學顯微鏡的解析度是有極限的，解析度...,光學顯微鏡,None,"[光學顯微鏡, 10]",0.001619
7,光學顯微鏡是以電磁學原理來將不可見或難見的微小物放大至肉眼可見的儀器。,光學顯微鏡的解析度受到光波長的限制，一般不超過0.3微米。,光學顯微鏡,None,"[光學顯微鏡, 11]",0.002479
8,光學顯微鏡是以電磁學原理來將不可見或難見的微小物放大至肉眼可見的儀器。,假如顯微鏡使用紫外線作爲光源或物體被放在油中的話，解析度還可以得到提高。,光學顯微鏡,None,"[光學顯微鏡, 12]",0.000652
9,光學顯微鏡是以電磁學原理來將不可見或難見的微小物放大至肉眼可見的儀器。,光學顯微鏡依樣品的不同可分爲反射式和透射式。,光學顯微鏡,None,"[光學顯微鏡, 15]",0.001647


notebook3
## PART 3. Claim verification

import libs

In [27]:
import pickle
from pathlib import Path
from typing import Dict, Tuple

import numpy as np
import pandas as pd
from pandarallel import pandarallel
from tqdm.auto import tqdm

import torch
from sklearn.metrics import accuracy_score
from utils import (
    generate_evidence_to_wiki_pages_mapping,
    jsonl_dir_to_df,
    load_json,
)


pandarallel.initialize(progress_bar=True, verbose=0, nb_workers=4)

Global variables

In [28]:
LABEL2ID: Dict[str, int] = {
    "supports": 0,
    "refutes": 1,
    "NOT ENOUGH INFO": 2,
}
ID2LABEL: Dict[int, str] = {v: k for k, v in LABEL2ID.items()}


Preload wiki database (same as part 2.)

In [29]:
wiki_pages = jsonl_dir_to_df("data/wiki-pages")
mapping = generate_evidence_to_wiki_pages_mapping(wiki_pages,)
del wiki_pages

Reading and concatenating jsonl files in data/wiki-pages
Generate parse mapping


Transform to id to evidence_map mapping


In [30]:
print("wiki_pages")

wiki_pages


In [31]:
def join_with_topk_evidence(
    df: pd.DataFrame,
    mapping: dict,
    mode: str = "train",
    topk: int = 5,
) -> pd.DataFrame:
    """join_with_topk_evidence join the dataset with topk evidence.

    Note:
        After extraction, the dataset will be like this:
               id     label         claim                           evidence            evidence_list
        0    4604  supports       高行健...     [[[3393, 3552, 高行健, 0], [...  [高行健 （ ）江西赣州出...
        ..    ...       ...            ...                                ...                     ...
        945  2095  supports       美國總...  [[[1879, 2032, 吉米·卡特, 16], [...  [卸任后 ， 卡特積極參與...
        停各种战争及人質危機的斡旋工作 ， 反对美国小布什政府攻打伊拉克...

        [946 rows x 5 columns]

    Args:
        df (pd.DataFrame): The dataset with evidence.
        wiki_pages (pd.DataFrame): The wiki pages dataframe
        topk (int, optional): The topk evidence. Defaults to 5.
        cache(Union[Path, str], optional): The cache file path. Defaults to None.
            If cache is None, return the result directly.

    Returns:
        pd.DataFrame: The dataset with topk evidence_list.
            The `evidence_list` column will be: List[str]
    """

    # format evidence column to List[List[Tuple[str, str, str, str]]]
    if "evidence" in df.columns:
        df["evidence"] = df["evidence"].parallel_map(
            lambda x: [[x]] if not isinstance(x[0], list) else [x]
            if not isinstance(x[0][0], list) else x)

    print(f"Extracting evidence_list for the {mode} mode ...")
    df["evidence_list"] = df["predicted_evidence"].parallel_map(lambda x: [
            mapping.get(evi_id, {}).get(str(evi_idx), "")
            for evi_id, evi_idx in x  # for each evidence list
        ][:topk] if isinstance(x, list) else [])
    print(df["evidence_list"][:5])

    return df

### Step 1. Setup training environment

Hyperparams

In [32]:
#@title  { display-mode: "form" }

EVIDENCE_TOPK = 5  #@param {type:"integer"}

Experiment Directory

### Step 2. Concat claim and evidences
join topk evidence

### Step 3. Training

Prevent CUDA out of memory

In [33]:
torch.cuda.empty_cache()

Training (30 mins)

In [34]:
def list_to_string(lst):
    # 將 list 轉換為 string，並刪除所有空格
    return "[PAD]".join(str(item).replace(' ', '') for item in lst)

### Step 4. Make your submission

In [35]:
from autogluon.multimodal import MultiModalPredictor
predictor = MultiModalPredictor.load(step3_model)
TEST_DATA = load_json("data/test_doc5sent5.jsonl")
TEST_PKL_FILE = Path("data/test_doc5sent5.pkl")

test_df = join_with_topk_evidence(
        pd.DataFrame(TEST_DATA),
        mapping,
        mode="eval",
        topk=EVIDENCE_TOPK,
    )
test_df.to_pickle(TEST_PKL_FILE, protocol=4)


/opt/conda/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
Load pretrained checkpoint: /workspace/main/AICUP-2023-Truth-main/step3_base_upload/model.ckpt


Extracting evidence_list for the eval mode ...


0    [光學顯微鏡 （ Optical microscope 、 Light microscope...
1    [家蠶 （ 學名 ： Bombyx mori ） 是鱗翅目蠶蛾科家蠶蛾屬的完全變態昆蟲 ， ...
2    [綠山城縣  ， 是波蘭的縣份 ， 位於該國西部 ， 由盧布斯卡省負責管轄 ， 首府設於綠山...
3    [《 魂斷藍橋 》 （ Waterloo Bridge ） 是美國黑白電影 ， 由米高梅電影...
4    [2015年以 《 刺客聶隱娘 》 獲得第68屆坎城影展最佳導演獎及第52屆金馬獎最佳導演獎...
Name: evidence_list, dtype: object


In [36]:
test_df_org = test_df.copy()
test_df

,id,claim,predicted_pages,predicted_evidence,evidence_list
0,5208,光學顯微鏡是以電磁學原理來將不可見或難見的微小物放大至肉眼可見的儀器。,"[光學顯微鏡, 物, 顯微鏡, 肉眼, 電磁學]","[[光學顯微鏡, 0], [顯微鏡, 0], [光學顯微鏡, 3], [光學顯微鏡, 9],...",[光學顯微鏡 （ Optical microscope 、 Light microscope...
1,1019,產絲的蠶或產蜜的蜜蜂爲提供間接經濟利益的昆蟲。,"[蠶, 昆蟲]","[[蠶, 0], [昆蟲, 22], [昆蟲, 23], [蠶, 4], [蠶, 5]]",[家蠶 （ 學名 ： Bombyx mori ） 是鱗翅目蠶蛾科家蠶蛾屬的完全變態昆蟲 ， ...
2,8514,波蘭西部的綠山城縣平均每平方公里的土地有0人。,"[綠山城縣, 人, 0, 波蘭, 土地, 西部]","[[綠山城縣, 0], [波蘭, 1], [波蘭, 0], [土地, 10], [土地, 8]]",[綠山城縣 ， 是波蘭的縣份 ， 位於該國西部 ， 由盧布斯卡省負責管轄 ， 首府設於綠山...
3,1874,VivienLeigh主演魂斷藍橋中的女配角。,"[配角, 藍橋, 橋, 魂斷藍橋]","[[魂斷藍橋, 0], [藍橋, 2], [藍橋, 1], [魂斷藍橋, 6], [魂斷藍橋...",[《 魂斷藍橋 》 （ Waterloo Bridge ） 是美國黑白電影 ， 由米高梅電影...
4,8352,侯孝賢改編自唐代文言文學的電影獲得金馬獎最佳劇情片獎。,"[劇情片, 文學, 文言文, 金馬獎, 電影, 侯孝賢]","[[侯孝賢, 2], [侯孝賢, 1], [侯孝賢, 0], [侯孝賢, 3], [侯孝賢,...",[2015年以 《 刺客聶隱娘 》 獲得第68屆坎城影展最佳導演獎及第52屆金馬獎最佳導演獎...
...,...,...,...,...,...
9033,12412,劉德華在中國星集團拍攝的最後一部電影爲大隻佬_(電影），之後他就結束與中國星集團14年賓主關係。,"[中國星集團, 電影, 佬, 大隻佬_(電影), 劉德華]","[[大隻佬_(電影), 8], [大隻佬_(電影), 0], [大隻佬_(電影), 5], ...",[該片是劉德華在中國星集團拍攝的最後一部電影 ， 之後便結束與中國星集團的14年賓主關係 ，...
9034,9611,視光學是利用光學儀器搭配侵入的方式來治療患者雙眼視覺異常。,"[患者, 眼, 雙眼視覺, 視光學, 光學儀器, 光學]","[[視光學, 0], [視光學, 1], [雙眼視覺, 18], [眼, 4], [光學, 0]]",[視光學 （ 英文 ： Optometry ） ， 是光學和眼科的結合 ， 運用光學儀器來檢...
9035,10538,婊子千層麪這首歌諷刺了印度音樂唱片公司「TSeries」，用來回應TSeries在訂閱數即將...,"[PewDiePie, 婊子千層麪]","[[婊子千層麪, 1], [婊子千層麪, 6], [婊子千層麪, 2], [婊子千層麪, 0...",[這首歌諷刺了印度音樂唱片公司 「 T - Series 」 ， 以回應T - Series...
9036,11757,兩棲動物由於在水中適應比不上腔棘魚，在陸地生存又落後羊膜動物，因此現今大多種類已絕種。,"[兩棲動物, 水, 陸地, 腔棘魚, 魚, 動物, 羊膜, 羊膜動物, 腔]","[[兩棲動物, 7], [兩棲動物, 8], [兩棲動物, 3], [兩棲動物, 6], [...",[早期兩棲動物在石炭紀繁盛一時 ， 分化出許多大型種類 ， 爲淡水和陸地上的頂級捕食者 ， ...


In [37]:
test_df = test_df[["claim","evidence_list"]]
test_df['evidence_list'] = test_df['evidence_list'].apply(list_to_string)

/tmp/ipykernel_1965/839020936.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['evidence_list'] = test_df['evidence_list'].apply(list_to_string)


In [38]:
test_df

,claim,evidence_list
0,光學顯微鏡是以電磁學原理來將不可見或難見的微小物放大至肉眼可見的儀器。,光學顯微鏡（Opticalmicroscope、Lightmicroscope）是一種利用光...
1,產絲的蠶或產蜜的蜜蜂爲提供間接經濟利益的昆蟲。,家蠶（學名：Bombyxmori）是鱗翅目蠶蛾科家蠶蛾屬的完全變態昆蟲，爲絲綢的主要原料來源...
2,波蘭西部的綠山城縣平均每平方公里的土地有0人。,"綠山城縣，是波蘭的縣份，位於該國西部，由盧布斯卡省負責管轄，首府設於綠山城，面積1,571平..."
3,VivienLeigh主演魂斷藍橋中的女配角。,《魂斷藍橋》（WaterlooBridge）是美國黑白電影，由米高梅電影公司於1940年出品...
4,侯孝賢改編自唐代文言文學的電影獲得金馬獎最佳劇情片獎。,2015年以《刺客聶隱娘》獲得第68屆坎城影展最佳導演獎及第52屆金馬獎最佳導演獎與金馬獎最...
...,...,...
9033,劉德華在中國星集團拍攝的最後一部電影爲大隻佬_(電影），之後他就結束與中國星集團14年賓主關係。,該片是劉德華在中國星集團拍攝的最後一部電影，之後便結束與中國星集團的14年賓主關係，加盟寰亞...
9034,視光學是利用光學儀器搭配侵入的方式來治療患者雙眼視覺異常。,視光學（英文：Optometry），是光學和眼科的結合，運用光學儀器來檢查眼睛的視覺功能，並...
9035,婊子千層麪這首歌諷刺了印度音樂唱片公司「TSeries」，用來回應TSeries在訂閱數即將...,這首歌諷刺了印度音樂唱片公司「T-Series」，以回應T-Series在訂閱數即將超過Pe...
9036,兩棲動物由於在水中適應比不上腔棘魚，在陸地生存又落後羊膜動物，因此現今大多種類已絕種。,早期兩棲動物在石炭紀繁盛一時，分化出許多大型種類，爲淡水和陸地上的頂級捕食者，但由於食性較爲...


In [39]:
predicted_label = predictor.predict(test_df)

Predicting: 0it [00:00, ?it/s]

Prediction

In [40]:
predict_dataset = test_df_org
predict_dataset["predicted_label"] = predicted_label
OUTPUT_FILENAME = "submission.jsonl"
predict_dataset[["id", "predicted_label", "predicted_evidence"]].to_json(
    OUTPUT_FILENAME,
    orient="records",
    lines=True,
    force_ascii=False,
)

將預測 SUPPORTS 或是 REFUTES 但 predicted_evidence 是空的更改成 NOT ENOUGH INFO

In [41]:
import pandas as pd 
from utils import load_json

data = load_json(OUTPUT_FILENAME)
# data predicted_evidence is 0 change this label
for item in data:
    if item['predicted_label'] == 'NOT ENOUGH INFO':
        continue
    length = len(item['predicted_evidence'])
    if length == 0:
        print(item)
        item['predicted_label'] = 'NOT ENOUGH INFO'
# save this json 

df = pd.DataFrame(data)
df.to_json(OUTPUT_FILENAME+"_Del", orient='records', lines=True, force_ascii=False)

{'id': 533, 'predicted_label': 'supports', 'predicted_evidence': []}
{'id': 13926, 'predicted_label': 'supports', 'predicted_evidence': []}
{'id': 526, 'predicted_label': 'supports', 'predicted_evidence': []}
{'id': 17223, 'predicted_label': 'refutes', 'predicted_evidence': []}
